# Código 15: Iluminação


PS TEM QUE CORRIGIR OS COMENTÁRIOS DA MALHA NO LOAD FILE

## Código pré loop principal

### Inicialização do glfw e criação da janela

**Conceitos iniciais**

* Fonte de luz: qualquer objeto que emite energia brilhante
* Luz branca é a combinação das cores do espectro
* Toda superfície absorve algumas cores e reflete outras cores
* A cor da superfície é a cor refletida (ou seja, não absorvida) e visível aos nossos olhos
* Modelos de iluminação: representação matemática ou algorítmica que descreve como a luz interage com superfícies tridimensionais para criar uma imagem visual. Exemplo:
    * Luz (branca) = RGB (1.0, 1.0, 1.0)
    * Objeto = RGB (1.0, 0.5, 0.31)
    * Objeto x Luz = RGB (1.0, 0.5, 0.31)
* Fonte de luz pontual: fonte de luz que é considerada como uma fonte infinitesimalmente pequena e concentrada em um único ponto no espaço

**Tipos de Iluminação**

* **Luz Ambiente**: iluminação geral presente em um ambiente, resultante da luz, sem uma fonte específica, que é dispersa em várias direções após interagir com superfícies e objetos. É responsável por garantir que objetos e cenários sejam visíveis, mesmo em áreas não diretamente iluminadas por fontes de luz diretas
* **Reflexão Difusa**: quando uma luz incide sobre uma superfície áspera ou irregular e é dispersa em muitas direções diferentes, produzindo iluminação suave e uniforme
* **Reflexão Especular**: quando uma luz incide sobre uma superfície lisa e reflete em um ângulo específico, seguindo a lei da reflexão, resultando em reflexos brilhantes e intensos que reproduzem a imagem da fonte de luz. Superfícies altamente polidas, como espelhos ou metais brilhantes, são exemplos de materiais que exibem reflexão especular perfeita. Em contexto de Computação Gráfica, podemos definir melhor específicamente como reflexão da luz incidente em uma área concentrada ao redor de um ângulo.

**Modelos e matemática**

Para representar essas três maneiras de iluminação, criou-se modelos com base nas leis físicas que aproximam o cenário da realidade. Porém, estes modelos não são exatos e não tem objetivo de serem, uma vez que o custo para computar um modelo realista é extremamente alto.

* **Luz Ambiente**: cada objeto irá refletir luz conforme ($i$) suas propriedades e a ($ii$) intensidade da luz

    ($i$) $k_a$: coeficiente de reflexão ambiente do objeto, com $k_a \in [0,1]$

    ($ii$) $I_a$: intensidade da luz ambiente, com $I_a \in [0,1]$
    
* **Reflexão Difusa**: quantidade de luz incidente depende do ângulo de incidência $\theta$ entre a direção da luz incidente e a normal da superfıcie. Quanto menor, mais forte é a reflexão e vice versa. Essa fato explica a dependência da reflexão pelo $cos(\theta)$. Considere:
    * $\overrightarrow{P_{source}}$: posição da fonte de luz
    * $\overrightarrow{P_{surf}}$: posição da surpefície
    * $\overrightarrow{L}$: vetor unitário que representa a direção da Luz. É dado por $$\overrightarrow{L} = \frac{\overrightarrow{P_{source}} - \overrightarrow{P_{surf}}}{|\overrightarrow{P_{source}} - \overrightarrow{P_{surf}}|}$$
    
    * $\overrightarrow{N}$: vetor unitário normal à superfície. Um vértice sozinho não forma superfície, então $\overrightarrow{N}$ pode ser calculado considerando a superfície local formada por vértices vizinhos. Em termos práticos, um objeto será importado em .obj e ele terá várias faces. O arquivo .obj indicará quais são as normais das faces desse objeto. Essas normais serão usadas para os cálculos. Vértices de uma mesma face usará a mesma normal, exceto se o arquivo .obj especificar algo diferente. Em geral, normais dos vértices das arestas também são especificados.
    
    ![Normal](15_Iluminacao02.png)
    
    * $cos(\theta) = \overrightarrow{N} \cdot \overrightarrow{L}$. Note que o objeto será iluminado apenas se $0.0 \leq \theta \leq 90º$. Se $\theta < 0.0$, luz estará atrás da superfície.

    ![N, L e theta](15_Iluminacao01.png)
    * Modelo final para reflexão difusa e ambiente: $$I_{diff} =
\begin{cases}
k_aI_a + k_dI_l(\overrightarrow{N}\cdot\overrightarrow{L}) & \text{ se } \overrightarrow{N}\cdot\overrightarrow{L} > 0 \\ 
k_aI_a  & \text{ se } \overrightarrow{N}\cdot\overrightarrow{L} \leq 0
\end{cases}$$
Onde,
        * $k_a$: coeficiente de reflexão ambiente do objeto, com $k_a \in [0,1]$
        * $I_a$: intensidade da luz ambiente, com $I_a \in [0,1]$
        * $k_d$: coeficiente de reflexão difusa do objeto, com $k_d \in [0,1]$
        * $I_l$: intensidade da luz pontual, com $I_l \in [0,1]$
        * $\overrightarrow{N}$: vetor unitário normal à superfície
        * $\overrightarrow{L}$: vetor unitário que representa a direção da Luz

    ![k_a e k_d](15_Iluminacao03.png)

* **Reflexão Especular**: essa reflexão depende do campo de reflexão especular. Para entender melhor esse efeito, definamos:
    * $\overrightarrow{R}$: vetor unitário que representa a direção da reflexão especular
    * $\overrightarrow{V}$: vetor unitário que representa a direção do observador
    * $\phi$: ângulo entre $\overrightarrow{R}$ e $\overrightarrow{V}$

    ![R e V](15_Iluminacao04.png)

    * **Campo de reflexão especular**: superfícies perfeitamente brilhantes refletem de forma organizada os raios de luz. Ou seja, um raio de luz que incide em uma superfície perfeitamente brilhante sempre vai refletir a partir de um ângulo $\theta$ da normal. Porém, superfícies menos brilhante refletem com um ângulo aproximadamente $\theta$, mas não $\theta$ exatamente. O ângulo varia, mas a variação é pequena. Superfície foscas refletem de forma completamente desorganizada e esse ângulo costuma variar muito. Essa área que é definida por essa variação é chamada de campo de reflexão especular. Superfícies brilhantes tem um campo menor de reflexão especular, enquanto que superfícies foscas tem um campo maior de reflexão especular.

    ![Campo de Reflexão Especular](15_Iluminacao05.png)

    * **Modelo de Phong**: modelo que modela o campo de reflexão especular. Define que a intensidade de reflexão especular depende de $cos^{n_s}(\phi)$, onde $n_s$ é expoente de reflexão especular. Quanto mais brilhante a superfície, mais $n_s \rightarrow \infty$. O modelo de matemático é definido como: $$I_{l,spec} =
\begin{cases}
k_sI_l(\overrightarrow{V}\cdot\overrightarrow{R})^{n_s} & \text{ se } \overrightarrow{V}\cdot\overrightarrow{R} > 0 \\ 
0.0 & \text{ se } \overrightarrow{V}\cdot\overrightarrow{R} \leq 0
\end{cases}$$
        Onde,
        * $k_s$: coeficiente de reflexão especular, com $k_s \in [0,1]$. Define o quão brilhante será a reflexão especular.
        * $n_s$: expoente de reflexão especular. Define o campo de reflexão especular.
        * $cos(\phi) = \overrightarrow{V}\cdot\overrightarrow{R}$

        ![Efeito do modelo de Phong](15_Iluminacao08.png)

        Porém, qual o valor de $\overrightarrow{V}$ e de $\overrightarrow{R}$?
        * $\overrightarrow{V} = \frac{\overrightarrow{P_{cam}} - \overrightarrow{P_{surf}}}{|\overrightarrow{P_{cam}} - \overrightarrow{P_{surf}}|}$
        * $|\overrightarrow{R}| = |\overrightarrow{N}|(2\overrightarrow{N}\cdot\overrightarrow{L}) - |\overrightarrow{L}|$:

        ![Calculo de R](15_Iluminacao06.png)
    

    * **Modelo de Phong simplificado**: utilizando vetor intermediário $\overrightarrow{H} = \frac{\overrightarrow{L} - \overrightarrow{V}}{|\overrightarrow{L} - \overrightarrow{V}|}$. Vide imagem a seguir. Motivação: para superfícies não planares, $\overrightarrow{N} \cdot \overrightarrow{H}$ requer menos cálculos do que $\overrightarrow{V} \cdot \overrightarrow{R}$, uma vez que o cálculo de $\overrightarrow{R}$ envolve saber a normal da superfície. Outro questão é que, se a posição da visão e da fonte de luz forem distantes, $\overrightarrow{V}$ e $\overrightarrow{L}$ serão constantes e $\overrightarrow{H}$ será constante também. Neste caso, se $\overrightarrow{N} \cdot \overrightarrow{H}$ for usado, tracaremos $cos(\phi$) por $cos(\alpha)$.

        ![Calculo de H](15_Iluminacao07.png)

**Modelo Final: Ambiente + Difusa + Especular**

$$ I = I_{diff} + I_{spec} = k_aI_a + k_dI_l(\overrightarrow{N} \cdot \overrightarrow{L}) + k_sI_l(\overrightarrow{N} \cdot \overrightarrow{H})^{n_s}$$

Onde, 

* $k_a$: coeficiente de reflexão ambiente do objeto, com $k_a \in [0,1]$
* $I_a$: intensidade da luz ambiente, com $I_a \in [0,1]$
* $k_d$: coeficiente de reflexão difusa do objeto, com $k_d \in [0,1]$
* $I_l$: intensidade da luz pontual, com $I_l \in [0,1]$
* $\overrightarrow{N}$: vetor unitário normal à superfície
* $\overrightarrow{L}$: vetor unitário que representa a direção da Luz
* $k_s$: coeficiente de reflexão especular, com $k_s \in [0,1]$
* $n_s$: expoente de reflexão especular
* $\overrightarrow{H}$: vetor unitário intermediário entre o vetor unitário que representa a direção da Lu e vetor unitário que representa a direção do observador

Observações:

* $\overrightarrow{N} \cdot \overrightarrow{L} \leq 0.0$: objeto será iluminado apenas pela luz ambiente
* $\overrightarrow{N} \cdot \overrightarrow{H} \leq 0.0$: não existirá reflexão especular

**Modelo Final: Múltiplas Fontes**

$$ I = I_{amb} + \sum_{l = 1}^{n} [I_{l,diff} + I_{l,spec}] = k_aI_a + \sum_{l = 1}^{n} I_l[k_d(\overrightarrow{N} \cdot \overrightarrow{L}) + k_s(\overrightarrow{N} \cdot \overrightarrow{H})^{n_s}]$$

**Coeficientes de reflexão RGB**

Quebrar a iluminação e os coeficientes de reflexão em cores RGB pode facilitar a modelagem do material da superfícies produzindo mais realismo. Desta maneira, teríamos:

* $I_l = (I_{lR}, I_{lG}, I_{lB})$
* $k_a = (k_{aR}, k_{aG}, k_{aB})$
* $k_d = (k_{dR}, k_{dG}, k_{dB})$
* $k_s = (k_{sR}, k_{sG}, k_{sB})$


In [91]:
#Bibliotecas
try:
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import random
    import glm
    from PIL import Image
except ImportError:
    !pip install glfw
    !pip install pyopengl
    !pip install numpy
    !pip install pyglm
    !pip install pillow
    import glfw
    from OpenGL.GL import *
    import numpy as np
    import math
    import random
    import glm
    from PIL import Image

#Sistema glfw
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE)

#get_dim_pos: retorna tamanho da tela e posição da tela
def get_dim_pos(per_width = 0.6, per_height = 0.6): 
    # Obtendo configurações do monitor
    monitores = glfw.get_monitors()
    monitor = monitores[0]
    video_mode = glfw.get_video_mode(monitor)
    WIDTH_WINDOW, HEIGHT_WINDOW = video_mode.size
    # Definindo proporção que se quer do monitor
    WIDTH_WINDOW : int = int(per_width*WIDTH_WINDOW)
    HEIGHT_WINDOW : int = int(per_height*HEIGHT_WINDOW)
    POSX_WINDOW : int = (video_mode.size[0] - WIDTH_WINDOW) // 2
    POSY_WINDOW : int = (video_mode.size[1] - HEIGHT_WINDOW) // 2
    return WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW

# Pega tamanho da tela e posição da tela
WIDTH_WINDOW, HEIGHT_WINDOW, POSX_WINDOW, POSY_WINDOW = get_dim_pos(0.6,0.6)
# Criando janela
TITLE: str = "Iluminação"
window = glfw.create_window(WIDTH_WINDOW, HEIGHT_WINDOW, TITLE, None, None)
glfw.set_window_pos(window, POSX_WINDOW, POSY_WINDOW)
glfw.make_context_current(window)

### Shaders: Vertex e Fragment

Relembrando os três principais tipos de qualificadores do GLSL:
* `attribute`: variáveis que representam dados de entrada específicos de cada vértice
* `varying`: historicamente dedicado para calcular valores intermediários entre superficies entre primitivas. Porém, com versões recentes, ele também é utilizado para variáveis que são declaradas no shader de vertice e são utilizadas no shader de fragmento. É uma notação alternativa para o conceito de `in` e `out`
* `uniform`: variáveis que permanecem constantes durante a execução de um shader

**SHADER VERTEX**

Relembrando variáveis já utilizadas em outros códigos:
* `attribute vec3 position`: coordenadas tridimensionais da posição de um vértice em um espaço 3D
* `attribute vec2 texture_coord`: representa as coordenadas de textura 2D associadas ao vértice
* `varying vec2 out_texture`: preenchida com as coordenadas de textura associadas a cada vértice, porém de forma interpolada automaticamente entre os vértices ao longo da primitiva durante o processo de renderização
* `uniform mat4 model`: matriz model
* `uniform mat4 view`: matriz view
* `uniform mat4 projection`: matriz projection


**Novas variáveis**:
* `attribute vec3 normals`: coordenadas tridimensionais da normal de um vértice
* `varying vec3 out_fragPos`: posição final do vértice no espaço. Ela é será usado como o vetor $\overrightarrow{P_{surf}}$ para o cálculo de $\overrightarrow{L}$ e $\overrightarrow{V}$ no shader de fragmento no contexto de modelagem de iluminação. Como só queremos a posição e não a renderização final, o valor desta variável é a posição multiplicada pela matriz model. A necessidade dela ser varying porque seu valor será automaticamente passado para o fragment shader, onde poderá ser utilizado conforme necessário
* `varying vec3 out_normal`: coordenadas tridimensionais finais da normal do vértice no espaço de mundo. A necessidade dela ser varying porque seu valor será automaticamente passado para o fragment shader, onde poderá ser utilizado conforme necessário


**SHADER FRAGMENT**

Relembrando variáveis já utilizadas em outros códigos:
* `varying vec2 out_texture`: coordenadas com valores interpolados dos vértices de textura vindos do shader de vértice
* `uniform sampler2D samplerTexture`: 
    * `sampler2D`: variável especial em GLSL usado para representar texturas 2D. Ela não armazena a textura de fato, mas possui um identificador para ela
    * `samplerTexture`: nome da variável
* `vec4 texture = texture2D(samplerTexture, out_texture)`: função usada para amostrar textura 2D. O primeiro argumento é o identificado e a segunda as coordenadas

**Novas variáveis Pré-Main**:
* `uniform vec3 lightPos`: coordenadas de posição da fonte de luz no espaço 3D. Representa $\overrightarrow{P_{source}}$ usado para calcular $\overrightarrow{L}$
* `vec3 lightColor = vec3(1.0, 1.0, 1.0)`: cor da luz. Neste caso, branca. Representa $I_a$, $I_l$ e $I_s$
* `uniform float ka`: coeficiente de reflexão ambiente $k_a$, que controla a intensidade da iluminação ambiente
* `uniform float kd`: coeficiente de reflexão difusa $k_d$, que controla a intensidade da iluminação difusa
* `uniform vec3 viewPos`: coordenadas da posição da câmera ou observador no espaço. Representa $\overrightarrow{P_{cam}}$ usado para calcular $\overrightarrow{V}$
* `uniform float ks`: coeficiente de reflexão especular $k_s$, que controla a intensidade da iluminação especular
* `uniform float ns`: expoente de reflexão especular $n_s$, que controla o tamanho do destaque especular
* `varying vec3 out_fragPos`: posição final do vértice no espaço. Valores vindos do shader de vértice. Representa vetor $\overrightarrow{P_{surf}}$ para o cálculo de $\overrightarrow{L}$ e $\overrightarrow{V}$
* `varying vec3 out_normal`: coordenadas tridimensionais finais da normal do vértice no espaço de mundo. Valores vindos do shader de vértice


**Novas variáveis Main**:

* REFLEXÃO AMBIENTE:
    * `vec3 ambient = ka * lightColor`: representa a parcela $k_aI_a$

* REFLEXÃO DIFUSA:
    * `vec3 norm = normalize(out_normal)`: transforma $\overrightarrow{N}$ em unitário
    * `vec3 lightDir = normalize(lightPos - out_fragPos)`: calcula $\overrightarrow{L} = \frac{\overrightarrow{P_{source}} - \overrightarrow{P_{surf}}}{|\overrightarrow{P_{source}} - \overrightarrow{P_{surf}}|}$
    * `dot(norm, lightDir)`: calcula $\overrightarrow{N} \cdot \overrightarrow{L}$
    * `float diff = max(dot(norm, lightDir), 0.0)`: truque para pegar valor $\overrightarrow{N} \cdot \overrightarrow{L}$ apenas no caso em que ele for positivo
    * `vec3 diffuse = kd * diff * lightColor`: representa parcela $k_dI_l(\overrightarrow{N} \cdot \overrightarrow{L})$

* REFLEXÃO ESPECULAR:
    * `vec3 viewDir = normalize(viewPos - out_fragPos)`: calcula $\overrightarrow{V} = \frac{\overrightarrow{P_{cam}} - \overrightarrow{P_{surf}}}{|\overrightarrow{P_{cam}} - \overrightarrow{P_{surf}}|}$
    * `vec3 reflectDir = normalize(reflect(-lightDir, norm))`: faz o cálculo de $\overrightarrow{R}$ automaticamente
    * `dot(viewDir, reflectDir)`: calcula $\overrightarrow{R} \cdot \overrightarrow{V}$
    * `max(dot(viewDir, reflectDir), 0.0)`: truque para pegar valor $\overrightarrow{R} \cdot \overrightarrow{V}$ apenas no caso em que ele for positivo
    * `float spec = pow(max(dot(viewDir, reflectDir), 0.0), ns)`: calcula $(\overrightarrow{R} \cdot \overrightarrow{V})^{n_s}$
    * `vec3 specular = ks * spec * lightColor`: representa parcela $k_sI_s(\overrightarrow{R} \cdot \overrightarrow{V})^{n_s}$

* MODELO DE PHONG:
    * `vec4((ambient + diffuse + specular),1.0)`: representa $k_aI_a + k_dI_d(\overrightarrow{N} \cdot \overrightarrow{L}) + k_sI_s(\overrightarrow{R} \cdot \overrightarrow{V})^{n_s}$
    * `vec4 result = vec4((ambient + diffuse + specular),1.0) * texture`: aplicação da iluminação na textura

In [92]:
#GLSL para Vertex Shader
vertex_code = """
        //POSICAO VERTICES, TEXTURA E NORMAL
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
        //VALORES REPASSADOS PARA O SHADER FRAGMENT
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;

        //MATRIZES DE TRANSFORMACAO 
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(  model * vec4(position, 1.0));
            out_normal = vec3( model *vec4(normals, 1.0));            
        }
        """

#GLSL para Fragment Shader
fragment_code = """
        // POSICAO DA FONTE DE LUZ E COR DA LUZ
        uniform vec3 lightPos; // define coordenadas de posicao da luz
        vec3 lightColor = vec3(1.0, 1.0, 1.0);
        
        // ILUMINACAO AMBIENTE E DIFUSA
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // ILUMINACAO ESPECULAR
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular

        // VARIAVEIS VINDOS DO SHADER VERTEX
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader

        // PARAMETRO DE TEXTURA
        uniform sampler2D samplerTexture;      
        
        void main(){
            // REFLEXAO AMBIENTE
            vec3 ambient = ka * lightColor;             
        
            // REFLEXAO DIFUSA
            vec3 norm = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir = normalize(lightPos - out_fragPos); // direcao da luz
            float diff = max(dot(norm, lightDir), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse = kd * diff * lightColor; // iluminacao difusa
            
            // REFLEXAO ESPECULAR
            vec3 viewDir = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir = normalize(reflect(-lightDir, norm)); // direcao da reflexao
            float spec = pow(max(dot(viewDir, reflectDir), 0.0), ns);
            vec3 specular = ks * spec * lightColor;             
            
            // MODELO DE PHONG
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse + specular),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;
        }
        """

### Solicitando espaço, compilando e linkando

In [93]:
#Requisitando slot para GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

#Associando os códigos aos espaços
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

#Compilando shader de vértice
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

#Compilando shader de fragmento
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

#Associadno programas compilados ao programa principal
glAttachShader(program, vertex)
glAttachShader(program, fragment)

#Linkagem do programa
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
#Tornando programa o atual
glUseProgram(program)

In [ ]:
# Função: carrega o arquivo Wavefront
# Entrada: nome do arquivo
# Saida: estrutura que armazena o elemento (vertices, textura e relations)
def load_model_from_file(file_path_name):
    # Arrays que armazenaram informações de coordenadas ou vetores
    vertices = []       # Posições dos vértices
    texture_coords = [] # Coordenada dos vértices de textura
    normals = []        # Coordenada que define os vetores normais
    relations = []      # Modelo que conecta, a partir da funções dadas no .obj, 
                        # vértices do objeto, vértices dde textura e os vetores normais

    # Não é utilizado, mas refere-se ao material do .obj
    material = None

    # Abre o arquivo obj (wavefront) para leitura
    for line in open(file_path_name, "r"): ## para cada linha do arquivo .obj
        # Se for comentário, ignore esta linha e use a próxima
        if line.startswith('#'): continue

        # Quebra a linha por espaço
        values = line.split()
        # Se não há informações na linha, ignore esta linha e use a próxima
        if not values: continue

        # Recupera as informações
        ### Armazena coordenadas dos vertices do elemento no vetor vertices
        if values[0] == 'v':
            vertices.append(values[1:4])
        ### Armazena vetor normais no vetor normals
        elif values[0] == 'vn':
            normals.append(values[1:4])
        ### Armazena coordenadas das texturas no vetor texture_coords
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])
        ### Define o material 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        ### Armazena informações sobre a construção das relations
        elif values[0] == 'f':
            # Declara vetores intermediários
            relation_vert = []
            relation_texture = []
            relation_normal = []
            # Para cada uma das triplas da linha que define a função
            for bloco in values[1:]:
                # Separa o elemento em vetor de elementos separando os números que são separados por /
                positions = bloco.split('/')
                # Adiciona o primeiro número no vetor relation_vert (que representa o número da linha que encontra-se um vértice)
                relation_vert.append(int(positions[0]))
                # Adiciona o terceiro número no vetor relation_normal (que representa o número da linha que encontra-se a normal para aquele vértice)
                relation_normal.append(int(positions[2]))
                # Se o vetor com elementos separados por / for maior ou igual que dois
                # Se o segundo número do elemento for maior do que zero
                if len(positions) >= 2 and len(positions[1]) > 0:
                    # Adicione o segundo número no vetor relation_texture (que representa o número da linha que encontra-se um vértice de textura da figura)
                    relation_texture.append(int(positions[1]))
                else:
                    # Se não for maior ou igual a dois ou não for maior que zero, coloque zero na textura
                    relation_texture.append(0)
            # Após conseguir, provavelmente, os três valores para vértice, os três valores para textura, os três valores para normal e o tipo de material, insira no vetor relations
            relations.append((relation_vert, relation_texture, relation_normal, material))

    # Armazena cada uma das partes do modelo
    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['relations'] = relations
    model['normals'] = normals

    return model

In [ ]:
#Função: associa id com a textura
#Entradas: o id que queremos associar e o caminho do arquivo .jpg
#Saida: não possui, apenas associa
def load_texture_from_file(texture_id, img_textura):
    #Definindo o id
    glBindTexture(GL_TEXTURE_2D, texture_id)
    #Alterando configurações paramétricas de textura
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    #Abre imagem
    img = Image.open(img_textura)
    #Captura as dimensões
    img_width = img.size[0]
    img_height = img.size[1]
    #Transforma imagem para um sequência de bytes em formato raw de arquivo
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #Carregando os dados da imagem
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

In [ ]:
# Função: retorna uma lista para coordenadas dos vertices do objeto, outra para as da textura e outra para as das normais
def get_vertices_textures_normals(model):
    vertices_list = []    
    textures_coord_list = []
    normals_list = []
    # Para cada um das relations (num_line(v), num_line(vt), num_line(vn) material)
    for relation in model['relations']:
        # Para cada um dos números que representa a linha do vértice
        for vertice_id in relation[0]: # Pega o valor a coordenada do vértice
            vertices_list.append( model['vertices'][vertice_id-1] )
        # Para cada um dos números que representa a linha da coordenada da textura
        for texture_id in relation[1]:  # Pega o valor a coordenada da textura
            textures_coord_list.append( model['texture'][texture_id-1] )
        # Para cada um dos números que representa a linha da coordenada da normal
        for normal_id in relation[2]:
            normals_list.append( model['normals'][normal_id-1] )
    return vertices_list, textures_coord_list, normals_list

In [ ]:
#Ativando texturas 2D
glEnable(GL_TEXTURE_2D)
#Gerando ids
num_textures = 10
textures = glGenTextures(num_textures)

#Carregando modelo
PATH_WAVE : str = 'objetos_wavefront'
CAIXA_PATH_OBJ : str = f'{PATH_WAVE}\caixa\caixa.obj'
CAIXA_PATH_JPG : str = f'{PATH_WAVE}\caixa\caixa.jpg'
modelo = load_model_from_file(CAIXA_PATH_OBJ)
#Carregando imagem
id_caixa = 0
load_texture_from_file(id_caixa, CAIXA_PATH_JPG)
#Carregando coordenadas de vertice e textura
vertices_list, textures_coord_list, normals_list = get_vertices_textures_normals(modelo)

In [ ]:
#Finaliza a modelagem dos dados de vértices
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list

#Finaliza a modelagem dos dados de texturas
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)])
textures['position'] = textures_coord_list

#Finaliza a modelagem dos dados das normais
normals = np.zeros(len(normals_list), [("position", np.float32, 3)])
normals['position'] = normals_list

In [ ]:
#Solicita dois buffers para GPU
buffer = glGenBuffers(3)

# Enviando dados de vértice
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

# Enviando dados de textura
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
#Subindo os dados de textura para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
#Encontrando informações de stride e offset das texturas
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "texture_coord" e habilitando
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
#Linkando dados ao atributo "texture_coord"
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)


# Enviando dados das normais
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
#Subindo os dados das normais para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
#Encontrando informações de stride e offset das normais
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "normals" e habilitando
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
#Linkando dados ao atributo "normals"
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

In [ ]:
def desenha_caixa():
    # aplica a matriz model
    angle = 0.0
    
    r_x = 0.0; r_y = 1.0; r_z = 0.0;
    
    # translacao
    t_x = 0.0; t_y = 0.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    
    #### define parametros de ilumincao do modelo
    ka = 0.1 # coeficiente de reflexao ambiente do modelo
    kd = 0.5 # coeficiente de reflexao difusa do modelo
    ks = 0.9 # coeficiente de reflexao especular do modelo
    ns = ns_inc # expoente de reflexao especular
    
    loc_ka = glGetUniformLocation(program, "ka") # recuperando localizacao da variavel ka na GPU
    glUniform1f(loc_ka, ka) ### envia ka pra gpu
    
    loc_kd = glGetUniformLocation(program, "kd") # recuperando localizacao da variavel kd na GPU
    glUniform1f(loc_kd, kd) ### envia kd pra gpu    
    
    loc_ks = glGetUniformLocation(program, "ks") # recuperando localizacao da variavel ks na GPU
    glUniform1f(loc_ks, ks) ### envia ks pra gpu        
    
    loc_ns = glGetUniformLocation(program, "ns") # recuperando localizacao da variavel ns na GPU
    glUniform1f(loc_ns, ns) ### envia ns pra gpu        

    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 0)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 0, 36) ## renderizando

### Manipulação dos espaços de dados

In [95]:
#Requisitando espaço de buffer para GPU
buffer = glGenBuffers(1)
#Tornando o buffer o buffer padrão de dados
glBindBuffer(GL_ARRAY_BUFFER, buffer)
#Subindo os dados de vértice para o buffer na GPU
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
#glBindBuffer(GL_ARRAY_BUFFER, buffer)

#Encontrando informações de stride e offset dos vértices
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
#Capturando posição do atributo "position" e habilitando. Também captura a posicao de loc_color e loc_mat
loc_position = glGetAttribLocation(program, "position")
loc_color = glGetUniformLocation(program, "color")
loc_mat = glGetUniformLocation(program, "mat_transformation")
glEnableVertexAttribArray(loc_position)
#Linkando dados ao atributo "position"
glVertexAttribPointer(loc_position, 3, GL_FLOAT, False, stride, offset)

### Eventos para modificar a posição da câmera

Aqui será necessário definir três importantes vetores.
1. Posição da câmera (`cameraPos`): indica a posição da câmera no cenário. Nesse caso, iniciaremos na coordenada `(0.0, 0.0, 1.0)`
2. Frente da câmera (`cameraFront`): direção para a qual a câmera está apontando. Iniciaremos apontando para o centro. Fazemos isso com `(0.0,  0.0, -1.0)`
3. Subida da câmera (`cameraUp`): direção do "para cima" da câmera. Neste caso, naturalmente é `(0.0,  1.0,  0.0)`.

É importante que todos os vetores, inclusive os resultantes das operações entre estes três, tenham módulo unitário. Desta maneira, conseguimos controlar a velocidade da mudança da posição através de uma variável defina apenas para isso: `cameraSpeed`. O módulo 1 pode ser tratado imediatamente antes da geração da matriz final.

**Movimentação para frente ou para trás:**

Para a câmera se mover para frente, basta que a posição da câmera ande na direção da frente de câmera. Podemos controlar a velocidade disso atráves da variável velocidade. Um ponto importante é caso a câmera esteja apontando para um outro lugar (na diagonal, por exemplo). Neste caso, como a direção está sendo controlado pelos eventos de cursor, a própria função cursor se resopnsabiliza de alterar o vetor `cameraFront` para garantir que ele fique sempre na mesma direção para onde a câmera aponta. Para andar para trás, basta fazer a operação de subtração.

**Movimentação para esquerda e direita:**

Para a câmera se mover para direita, basta que ande para a direção do produto vetorial entre a direção da frente e a da subida (que gerará um vetor para direita). O único ponto é que o vetor resultante deve ter módulo unitário. Para andar para esquerda, basta subtrair a posição deste vetor ao invés de somar. O módulo 1 pode ser tratado imediatamente antes da geração da matriz final.

**Evento de mouse**:

Para construir e entender a função evento de mouse, é preciso saber:
* Yaw: rotação em torno do eixo y. Define o "virar a câmera para direita ou esquerda"
* Pitch: rotação em torno do eixo x. Define o "virar a câmera para cima ou baixo"

Na função, calcula-se a variação de posição do mouse no eixo y e x para definir o yaw e o pitch respectivamente. Considerando apenas uma pequena parcela do valor (cerca de trinta porcento) da variação, podemos assumir, com certo erro, que yaw e pitch sejam de fato a rotação, em graus, da câmera. Com isso, podemos calcular a nova posição da frente de câmera utilizando algumas fórmuals matemáticas (que não é o foco).

In [96]:
# Posicao inicial da camera
cameraPos   = glm.vec3(0.0,  0.0,  1.0)
# Vetor responsável para apontar para frente
cameraFront = glm.vec3(0.0,  0.0, -1.0)
# Vetor auxiliar que aponta para cima em relação a camera
cameraUp    = glm.vec3(0.0,  1.0,  0.0)

# Funcao que captura evento do teclado
def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp
    
    # Componentes da câmera
    # Velocidade da camera
    cameraSpeed = 0.01
    # Ir para frente
    if key == 87 and (action==1 or action==2): # Tecla W
        cameraPos += cameraSpeed * cameraFront
    # Ir para trás
    if key == 83 and (action==1 or action==2): # Tecla S
        cameraPos -= cameraSpeed * cameraFront
    # Ir para esquerda
    if key == 65 and (action==1 or action==2): # Tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
    # Ir para direita    
    if key == 68 and (action==1 or action==2): # Tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed

# Variáveis auxiliar
# Flag para definir se eh a primeira vez que o mouse aparece na tela
firstMouse = True
# yaw: rotação no eixo y
yaw = -90.0 
# pitch: rotação no eixo x
pitch = 0.0
# Valores iniciais da última posição do mouse
lastX =  WIDTH_WINDOW/2
lastY =  HEIGHT_WINDOW/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    # Tratando caso de primeira aparição do mouse
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    # Calculos da variação
    xoffset = xpos - lastX
    yoffset = lastY - ypos
    # Atualizando valor da última posição
    lastX = xpos
    lastY = ypos
    # Calculando yam e pitch aproximadamente
    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity
    yaw += xoffset
    pitch += yoffset

    # Evitando que rotação extremas
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    # Fórmulas matemáticas para calcular o novo cameraFront
    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)

# Define função de evento para teclado
glfw.set_key_callback(window,key_event)
# Define função de evento para cursor
glfw.set_cursor_pos_callback(window, mouse_event)
# Seta posição do cursor
# glfw.set_cursor_pos(window, lastX, lastY)

### Definindo matrizes model, view e projection

**Mariz View**

Considerando que o vetor normal $\overrightarrow{N}$ é o vetor que parte do "target" e até a posição da câmera dado por $P_0 = (x_0, y_0, z_0)$, temos que:
* Câmera frente: $\overrightarrow{n} = \frac{\overrightarrow{N}}{|\overrightarrow{N}|} = (n_x,n_y,n_z)$
* Câmera subida intermediária:  $\overrightarrow{V}$ (geralmente dado como $(0.0,1.0,0.0)$)
* Câmera direita: $\overrightarrow{u} = \frac{\overrightarrow{V} \times \overrightarrow{n}}{|\overrightarrow{V} \times \overrightarrow{n}|} = (u_x,u_y,u_z)$
* Câmera subida: $\overrightarrow{v} = \overrightarrow{n} \times \overrightarrow{u} = (v_x, v_y, v_z)$ 
Por fim, a matriz view é: 
$View = \begin{bmatrix}
u_x & u_y & u_z & -u \cdot P_0\\ 
v_x & v_y & v_z & -v \cdot P_0\\  
n_x & n_y & n_z & -n \cdot P_0\\ 
0 & 0 & 0 & 1 
\end{bmatrix}$

**Mariz Projection**

Para construir, é importante destacar alguns termos desta imagem: 

![Termos da Projecao](14_Camera.png)
* Field of View (FOV): ângulo de visão da câmera
* Clipping window (Janela de Recorte): retângulo sobre o qual os objetos serão projetados
* Eixo $z_{view}$: é o eixo definido pelo vetor câmera frente
* Far Clipping Plane: plano imaginário que está localizado longe da câmera. Objetos que estão mais longes do que este plano não serão renderizados. Perpendicular ao eixo $z_{view}$.
* Near Clipping Plane: plano imaginário que está localizado perto da câmera. Objetos que estão mais próximos do que este plano não serão renderizados. Perpendicular ao eixo $z_{view}$.
* Frustum View Volume: tronco-piramidal que resulta da seção da pirâmide pelos planos paralelos Far e Near. Descreve o que será visível pela câmera.

Descrito isso, define-se:
* $\theta$: FOV
* $z_{near}$: distância da câmera até o plano de visão mais perto
* $z_{far}$: distância da câmera até o plano de visão mais distante
* $width$: largura da Clipping Window
* $height$: altura da Clipping Window
* $aspect$: relação $width/height$

Assim, a matriz projection é:
$Projection = \begin{bmatrix}
\frac{cot(\frac{\theta}{2})}{aspect} & 0 & 0 & 0\\ 
0 & cot(\theta/2) & 0z & 0\\  
0 & 0 & \frac{z_{near} + z_{far}}{z_{near} - z_{far}} & - \frac{2\cdot z_{near}\cdot z_{far}}{z_{near} - z_{far}} \\ 
0 & 0 & -1 & 0 
\end{bmatrix}$

Link para entender melhor o impacto de cada variável: https://webglfundamentals.org/webgl/frustum-diagram.html

In [97]:
# Matriz model: posição, orientação e escala do objeto no próprio espaço local
def model():
    # Neste exemplo, a matriz model não altera nenhum objeto
    mat_model = glm.mat4(1.0) # Matriz identidade
    mat_model = np.array(mat_model)    
    return mat_model

# Matriz view: posição e orientação da câmera no espaço 3D. Posiciona a cena em relação à câmera.
def view():
    global cameraPos, cameraFront, cameraUp
    # Parãmetros: posição da câmera, direção do target e câmera up. Câmera direita é calculado internamente pela função da biblioteca glm
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp)
    mat_view = np.array(mat_view)
    return mat_view

# Matriz Projection: transforma o volume de visualização 3D em um espaço 2D, levando em consideração fatores como a distância dos objetos à câmera
def projection():
    # Neste caso, definimos parâmetros estáticos, mas poderiam ser dinâmicos
    fov = glm.radians(45.0)
    aspect = WIDTH_WINDOW/HEIGHT_WINDOW
    near = 0.1
    far = 100.0
    mat_projection = glm.perspective(fov, aspect, near, far)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Criando funções de impressão de objetos

In [98]:
# Habilita 3D
glEnable(GL_DEPTH_TEST)

def desenha_cubo1():
    # DESENHANDO O CUBO 1 (vértices de 0 até 23)
    for i in range(0,24,4): # incremento de 4 em 4
        R = (i+1)/24
        G = (i+2)/24
        B = (i+3)/24
        glUniform4f(loc_color, R, G, B, 1.0) ### definindo uma cor qualquer com base no i
        glDrawArrays(GL_TRIANGLE_STRIP, i, 4)
    
def desenha_cubo2():
    # DESENHANDO O CUBO 2 (vértices de 24 até 47)
    for i in range(24,48,4): # incremento de 4 em 4
        R = (i+1)/48
        G = (i+2)/48
        B = (i+3)/48
        glUniform4f(loc_color, R, G, B, 1.0) ### definindo uma cor qualquer com base no i
        glDrawArrays(GL_TRIANGLE_STRIP, i, 4)

### Exibindo na tela

In [99]:
glfw.show_window(window)

## Loop principal

In [100]:
while not glfw.window_should_close(window):
    glfw.poll_events() 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)

    # Computando e enviando matrizes Model, View e Projection para a GPU
    mat_model = model()
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    
    # Desenhando objetos
    desenha_cubo1()
    desenha_cubo2()

    glfw.swap_buffers(window)

glfw.terminate()